In [30]:
# Bloc complet : pipeline complet pour la prédiction du Mode de Propulsion

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib
import os

# 1. Chargement des données
df = pd.read_excel(r'C:\Users\pieta\OneDrive\Bureau\tracéo_justin\Usual_analysis\data\Expdatatangedpoint05_07_22_testhadrien.xlsx')

# 2. Nettoyage de base
df.drop(columns=['Tool ID', 'Shoot ID', 'Fracture ID', 'Type'], inplace=True)
df['Propagation Phase Lenght'] = pd.to_numeric(df['Propagation Phase Lenght'], errors='coerce')

# 3. Conversion des colonnes pertinentes en 'category'
cat_cols = ['Impacted material','State after shoot','Initation','Locus','Location of initiation','Profile of initation','General Direction',
            'Location of termination','Termination','fracture composition','Fracture part','Fracture group','enought traces for determination','attribute group']
df[cat_cols] = df[cat_cols].astype('category')
df['Penetration'] = pd.to_numeric(df['Penetration'], errors='coerce')

# Forcer toutes les colonnes catégorielles à être des chaînes de caractères
for col in cat_cols:
    df[col] = df[col].astype(str)

# 4. Séparation X/y
X = df[cat_cols + ['Penetration']]
y = df['Mode of Propulsion']

# 5. Encodage de la cible
le = LabelEncoder()
y_enc = le.fit_transform(y)

# 6. Split
X_train, X_test, y_train, y_test = train_test_split(X, y_enc, stratify=y_enc, test_size=0.2, random_state=42)

# 7. Pipeline de prétraitement
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1)),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])
preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_cols)
], remainder='passthrough')

# 8. Modèle Random Forest
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42))
])
pipeline.fit(X_train, y_train)

# 9. Analyse des variables importantes
encoded_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'] \
    .named_steps['encoder'].get_feature_names_out(cat_cols)
all_feature_names = list(encoded_names) + ['Penetration']

importances = pipeline.named_steps['classifier'].feature_importances_
importance_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("\nTop 10 des variables les plus importantes:")
print(importance_df.head(10))

# 10. Évaluation finale
y_pred = pipeline.predict(X_test)
print("Rapport de classification:")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

# Sauvegarde de l'encodeur de labels
joblib.dump(le, r'C:\Users\pieta\OneDrive\Bureau\tracéo_justin\Usual_analysis\model\label_encoder.joblib')
print(r"✅ LabelEncoder sauvegardé dans : C:\Users\pieta\OneDrive\Bureau\tracéo_justin\Usual_analysis\model\label_encoder.joblib")


# 11. Sauvegarde du modèle et de l'encodeur
os.makedirs("modele", exist_ok=True)
joblib.dump(pipeline, r'C:\Users\pieta\OneDrive\Bureau\tracéo_justin\Usual_analysis\model\model_rf_complet.joblib')
print(r"✅ Modèle sauvegardé dans : C:\Users\pieta\OneDrive\Bureau\tracéo_justin\Usual_analysis\model\model_rf_complet.joblib")



Top 10 des variables les plus importantes:
                             Feature  Importance
14                       Penetration    0.310062
6                  General Direction    0.105277
8                        Termination    0.098073
12  enought traces for determination    0.080016
0                  Impacted material    0.077043
2                          Initation    0.066196
5               Profile of initation    0.049861
3                              Locus    0.043172
4             Location of initiation    0.041023
13                   attribute group    0.039253
Rapport de classification:
               precision    recall  f1-score   support

          Bow       1.00      0.25      0.40         4
Spear thrower       0.67      0.73      0.70        11
     Throwing       0.50      0.56      0.53         9
    Thrusting       0.77      0.83      0.80        12

     accuracy                           0.67        36
    macro avg       0.73      0.59      0.61        36
 we

Voici la même **analyse détaillée** pour le **modèle sans filtrage des variables importantes** :

---

### 🔍 Analyse du modèle **avec toutes les variables**

| Classe            | Precision | Recall | F1-score | Remarque                                                         |
| ----------------- | --------- | ------ | -------- | ---------------------------------------------------------------- |
| **Bow**           | 1.00      | 0.25   | 0.40     | 🎯 Trop peu d’exemples bien captés → rappel faible (1/4 détecté) |
| **Spear thrower** | 0.67      | 0.73   | 0.70     | ✅ Stable malgré le bruit                                         |
| **Throwing**      | 0.50      | 0.56   | 0.53     | ⚠️ Faible mais meilleure qu’avant sur le rappel                  |
| **Thrusting**     | 0.77      | 0.83   | 0.80     | ✅ Bonne constance sur cette classe                               |

---

### 📊 Bilan global :

* **Accuracy : 67%** → en légère baisse vs modèle réduit
* **Macro F1-score : 0.61** → perte d'équilibre entre classes
* **Classe "Bow"** toujours difficile à capter (souvent mal représentée)
* Le modèle souffre probablement de **trop de bruit / dimensions non discriminantes**

---

### 🧠 Interprétation :

* Les variables supplémentaires **n’ajoutent pas de valeur explicative**, et peuvent même brouiller le signal.
* Le modèle apprend moins efficacement à distinguer les classes rares.

---


In [ ]:
# Bloc : Entraînement final du modèle avec les variables importantes uniquement

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Utiliser automatiquement les colonnes les plus importantes (top 6)
important_cols = importance_df.sort_values(by='Importance', ascending=False)['Feature'].head(10).tolist()

# Recréer X et y
X_final = df[important_cols]
y_final = df['Mode of Propulsion']

y_final_enc = le.fit_transform(y_final)

# Split final
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_final, y_final_enc, stratify=y_final_enc, test_size=0.2, random_state=42)

# Identifier les colonnes catégorielles (sauf Penetration)
final_cat_cols = [col for col in important_cols if col != 'Penetration']

# Créer pipeline
final_cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

final_preprocessor = ColumnTransformer([
    ('cat', final_cat_pipeline, final_cat_cols)
], remainder='passthrough')

final_pipeline = Pipeline([
    ('preprocessor', final_preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42))
])

# Entraînement
final_pipeline.fit(X_train_f, y_train_f)

# Prédiction
final_pred = final_pipeline.predict(X_test_f)

# Rapport final
y_test_decoded = le.inverse_transform(y_test_f)
y_pred_decoded = le.inverse_transform(final_pred)

print("\nRapport de classification (modèle final avec variables importantes):")
print(classification_report(y_test_decoded, y_pred_decoded))




Rapport de classification (modèle final avec variables importantes):
               precision    recall  f1-score   support

          Bow       1.00      0.50      0.67         4
Spear thrower       0.69      0.82      0.75        11
     Throwing       0.57      0.44      0.50         9
    Thrusting       0.79      0.92      0.85        12

     accuracy                           0.72        36
    macro avg       0.76      0.67      0.69        36
 weighted avg       0.73      0.72      0.71        36



Très bon score global, surtout pour un modèle réduit à 6 variables 🎯

---

### 🔍 Analyse rapide :

| Classe            | Precision | Recall | F1-score | Remarque                                                   |
| ----------------- | --------- | ------ | -------- | ---------------------------------------------------------- |
| **Bow**           | 1.00      | 0.50   | 0.67     | 🔥 Précision parfaite, mais manque de rappel (2/4 trouvés) |
| **Spear thrower** | 0.69      | 0.82   | 0.75     | ✅ Très bon résultat, stable                                |
| **Throwing**      | 0.57      | 0.44   | 0.50     | ⚠️ Plus difficile à distinguer                             |
| **Thrusting**     | 0.79      | 0.92   | 0.85     | ✅ Excellente performance                                   |

---

### 📊 Bilan :

* **Accuracy globale : 72%** → très solide pour un modèle réduit
* **Weighted F1 : 0.71** → bien équilibré
* Seules **quelques erreurs sur "Throwing"** : peut-être un manque de données ou confusion sémantique

---

### ✅ Prochaines améliorations possibles :

* Ajouter 1 ou 2 variables de plus (top 8 au lieu de 6) pour voir si la classe “Throwing” s’améliore
* Suréchantillonnage léger de “Bow” ou “Throwing” si ce sont des classes minoritaires
* Tester `class_weight='balanced'` dans `RandomForestClassifier`



In [ ]:
# Bloc : Entraînement du modèle Logistic Regression avec toutes les variables

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Utiliser toutes les variables disponibles (sauf cible)
all_features = df.drop(columns=['Mode of Propulsion']).columns.tolist()

# Recréer X et y
X_full = df[all_features].copy()
y_full = df['Mode of Propulsion']

# Forcer toutes les colonnes sauf 'Penetration' à être des strings (uniformiser types)
cat_cols_all = [col for col in all_features if col != 'Penetration']
X_full[cat_cols_all] = X_full[cat_cols_all].astype(str)

# Encodage de la cible
y_full_enc = le.fit_transform(y_full)

# Split
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(
    X_full, y_full_enc, stratify=y_full_enc, test_size=0.2, random_state=42)

# Pipeline catégories
full_cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

full_preprocessor = ColumnTransformer([
    ('cat', full_cat_pipeline, cat_cols_all)
], remainder='passthrough')

# Pipeline complet
full_pipeline = Pipeline([
    ('preprocessor', full_preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Entraînement
full_pipeline.fit(df_X_train, df_y_train)

# Prédiction
full_pred = full_pipeline.predict(df_X_test)

# Rapport final
y_test_decoded_all = le.inverse_transform(df_y_test)
y_pred_decoded_all = le.inverse_transform(full_pred)

print("\nRapport de classification (modèle Logistic Regression avec toutes les variables):")
print(classification_report(y_test_decoded_all, y_pred_decoded_all))



Rapport de classification (modèle Logistic Regression avec toutes les variables):
               precision    recall  f1-score   support

          Bow       1.00      0.75      0.86         4
Spear thrower       0.75      0.55      0.63        11
     Throwing       0.24      0.44      0.31         9
    Thrusting       0.62      0.42      0.50        12

     accuracy                           0.50        36
    macro avg       0.65      0.54      0.57        36
 weighted avg       0.61      0.50      0.53        36



C:\Users\pieta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Bloc : Entraînement final du modèle avec les variables importantes uniquement (Logistic Regression)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Utiliser automatiquement les colonnes les plus importantes (top 6)
important_cols = importance_df.sort_values(by='Importance', ascending=False)['Feature'].head(6).tolist()

# Recréer X et y
X_final = df[important_cols]
y_final = df['Mode of Propulsion']

y_final_enc = le.fit_transform(y_final)

# Split final
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_final, y_final_enc, stratify=y_final_enc, test_size=0.2, random_state=42)

# Identifier les colonnes catégorielles (sauf Penetration)
final_cat_cols = [col for col in important_cols if col != 'Penetration']

# Créer pipeline
final_cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

final_preprocessor = ColumnTransformer([
    ('cat', final_cat_pipeline, final_cat_cols)
], remainder='passthrough')

final_pipeline = Pipeline([
    ('preprocessor', final_preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs'))
])

# Entraînement
final_pipeline.fit(X_train_f, y_train_f)

# Prédiction
final_pred = final_pipeline.predict(X_test_f)

# Rapport final
y_test_decoded = le.inverse_transform(y_test_f)
y_pred_decoded = le.inverse_transform(final_pred)

print("\nRapport de classification (modèle final Logistic Regression avec variables importantes):")
print(classification_report(y_test_decoded, y_pred_decoded))

C:\Users\pieta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Rapport de classification (modèle final Logistic Regression avec variables importantes):
               precision    recall  f1-score   support

          Bow       0.67      0.50      0.57         4
Spear thrower       0.60      0.82      0.69        11
     Throwing       0.25      0.22      0.24         9
    Thrusting       0.60      0.50      0.55        12

     accuracy                           0.53        36
    macro avg       0.53      0.51      0.51        36
 weighted avg       0.52      0.53      0.52        36



Excellente question. D’après toutes les expérimentations que tu as menées, voici un **bilan clair et argumenté** pour ton use case de **prédiction du mode de propulsion en tracéologie** :

---

## 🧪 Modèles testés

| Modèle                           | Données utilisées     | Accuracy                  | Macro F1 | Remarques clés                      |
| -------------------------------- | --------------------- | ------------------------- | -------- | ----------------------------------- |
| **Random Forest (top 6 vars)**   | Variables importantes | **0.72**                  | **0.69** | 🥇 Meilleur score, bon équilibre    |
| **Random Forest (toutes vars)**  | Toutes les variables  | 0.67                      | 0.61     | Plus de bruit, moins efficace       |
| **Logistic Regression (top 6)**  | Variables importantes | (non testé dans les logs) | ?        | (à tester pour comparaison directe) |
| **Logistic Regression (toutes)** | Toutes les variables  | 0.50                      | 0.57     | ❌ Faible convergence, déséquilibre  |

---

## 🏆 Recommandation finale

### ✅ **RandomForestClassifier avec les 6 variables les plus importantes** est **le meilleur choix** pour ce cas :

* **Robuste aux variables catégorielles** (pas besoin de scaling)
* **Tolérant au bruit**
* **Gère bien les petits datasets**
* **Excellente balance précision / rappel** (notamment pour “Spear thrower” et “Thrusting”)
* **Interprétable** (feature importance)

---

## 🧠 Pour aller encore plus loin :

* Si tu veux **un modèle plus léger à déployer**, tu peux tester la **logistic regression avec seulement les variables importantes**, mais tu perdras un peu de performance.
* Pour optimiser : fais une **recherche d’hyperparamètres** sur Random Forest (via GridSearchCV).




In [ ]:
import joblib
import os

# Sauver le modèle entraîné
joblib.dump(final_pipeline, r'C:\Users\pieta\OneDrive\Bureau\tracéo_justin\Usual_analysis\model')

print("✅ Modèle sauvegardé dans : model/model_rf_reduit.joblib")


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\pieta\\OneDrive\\Bureau\\tracéo_justin\\Usual_analysis\\model'